# CFFI - C Foreign Function Interface


# Goal: call C code from Python without learning a 3rd language

 * SWIG requires to you to learn some additional syntax to create interface files


# Two Modes

 * Application Binary Interface (ABI)
 * Application Progamming Interface (API) -- **Preferred**


# ABI Example

Calls an *existing* C library.  In this case, we'll create it.  First, write a C file:


In [4]:
%%file mkl_multiply.c
#include "mkl.h"

void mkl_multiply(const int n, const double *x, const double *y, double *result){
    const MKL_INT n_ = (const MKL_INT) n;
    vdMul(n, x, y, result);
}

Overwriting mkl_multiply.c


Compile the C file into a shared library


In [5]:
%%bash
gcc mkl_multiply.c -shared -o mkl_multiply.so -I${CONDA_PREFIX}/include -L${CONDA_PREFIX}/lib  -lmkl_rt 

In [6]:
%ls mkl_multiply*

mkl_multiply.c   mkl_multiply.so*


# ABI Example (cont.)

Import FFI, load library, and define functions to be shared between Python and C


In [7]:
from cffi import FFI
ffi = FFI()
mkl = ffi.dlopen('mkl_multiply.so')
ffi.cdef("""
         void mkl_multiply(const int n, const double *x, const double *y, double *result);
""")

Import numpy, create some arrays, and cast them to C data types.


In [10]:
import numpy as np

N = 100

x = np.arange(N, dtype=np.double)
y = np.arange(N, dtype=np.double)
result = np.empty_like(x)

x_ = ffi.cast("const double*", ffi.from_buffer(x))
y_ = ffi.cast("const double*", ffi.from_buffer(y))
result_ = ffi.cast("double *", ffi.from_buffer(result))


# ABI Example (cont.)

Test the function


In [11]:
%%timeit
mkl.mkl_multiply(N,  x_, y_, result_)

577 ns ± 95 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Compare the numpy


In [12]:
%timeit x*y

520 ns ± 25.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Compare to pure Python


In [13]:
%%timeit
for i in range(N):
    result[i] = x[i] * y[i]

35.2 µs ± 3.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# API Example

Create a "builder" Python script that includes the C source and the shared Python/C definitions, that is compiled into a single wrapper.


In [14]:
%%file mkl_multiply_builder.py
import os
from cffi import FFI
ffibuilder = FFI()

ffibuilder.set_source("mkl_multiply",
   r"""
   #include "mkl.h"

   void mkl_multiply(const int n, const double *x, const double *y, double *result){
        const MKL_INT n_ = (const MKL_INT) n;
        vdMul(n_, x, y, result);
   }
   """, 
   library_dirs=[os.environ['CONDA_PREFIX'] + '/lib'],
   include_dirs=[os.environ['CONDA_PREFIX'] + '/include'],
   libraries=['mkl_rt'])

ffibuilder.cdef("""
    void mkl_multiply(const int n, const double *x, const double *y, double *result);
""")

if __name__ == "__main__":
    ffibuilder.compile(verbose=True)

Writing mkl_multiply_builder.py


# API Example (cont.)

Run the builder script


In [15]:
%run mkl_multiply_builder.py

generating ./mkl_multiply.c
the current directory is '/Users/john/Documents/HPC_Course/Spring2018/lectures'
running build_ext
building 'mkl_multiply' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/john/miniconda3/envs/PGE383/include -arch x86_64 -I/Users/john/miniconda3/envs/PGE383/include -arch x86_64 -I/Users/john/miniconda3/envs/PGE383/include -I/Users/john/miniconda3/envs/PGE383/include/python3.6m -c mkl_multiply.c -o ./mkl_multiply.o
gcc -bundle -undefined dynamic_lookup -L/Users/john/miniconda3/envs/PGE383/lib -arch x86_64 -L/Users/john/miniconda3/envs/PGE383/lib -arch x86_64 -arch x86_64 ./mkl_multiply.o -L/Users/john/miniconda3/envs/PGE383/lib -lmkl_rt -o ./mkl_multiply.cpython-36m-darwin.so


In [16]:
%ls mkl_multiply*

mkl_multiply.c                      mkl_multiply.so*
mkl_multiply.cpython-36m-darwin.so* mkl_multiply_builder.py
mkl_multiply.o


# API Example (cont.)

Import the the library.  Notice there is no call to `dlopen()`.


In [17]:
from mkl_multiply import lib as mkl

Test the function.


In [18]:
%%timeit
mkl.mkl_multiply(N, x_, y_, result_)

381 ns ± 28.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
result

array([0.000e+00, 1.000e+00, 4.000e+00, 9.000e+00, 1.600e+01, 2.500e+01,
       3.600e+01, 4.900e+01, 6.400e+01, 8.100e+01, 1.000e+02, 1.210e+02,
       1.440e+02, 1.690e+02, 1.960e+02, 2.250e+02, 2.560e+02, 2.890e+02,
       3.240e+02, 3.610e+02, 4.000e+02, 4.410e+02, 4.840e+02, 5.290e+02,
       5.760e+02, 6.250e+02, 6.760e+02, 7.290e+02, 7.840e+02, 8.410e+02,
       9.000e+02, 9.610e+02, 1.024e+03, 1.089e+03, 1.156e+03, 1.225e+03,
       1.296e+03, 1.369e+03, 1.444e+03, 1.521e+03, 1.600e+03, 1.681e+03,
       1.764e+03, 1.849e+03, 1.936e+03, 2.025e+03, 2.116e+03, 2.209e+03,
       2.304e+03, 2.401e+03, 2.500e+03, 2.601e+03, 2.704e+03, 2.809e+03,
       2.916e+03, 3.025e+03, 3.136e+03, 3.249e+03, 3.364e+03, 3.481e+03,
       3.600e+03, 3.721e+03, 3.844e+03, 3.969e+03, 4.096e+03, 4.225e+03,
       4.356e+03, 4.489e+03, 4.624e+03, 4.761e+03, 4.900e+03, 5.041e+03,
       5.184e+03, 5.329e+03, 5.476e+03, 5.625e+03, 5.776e+03, 5.929e+03,
       6.084e+03, 6.241e+03, 6.400e+03, 6.561e+03, 

# More Information

 * [CFFI Documentation](https://cffi.readthedocs.io/en/latest/)


In [20]:
%%javascript
function hideElements(elements, start) {
    for(var i = 0, length = elements.length; i < length;i++) {
        if(i >= start) {
            elements[i].style.display = "none";
        }
    }
}

var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)

<IPython.core.display.Javascript object>